In [1]:
import json
import numpy
import pandas
import pathlib
import pydash
import requests
import tqdm

acmi_path = pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'json' / 'works'
acmi_files = [filename for filename in acmi_path.iterdir() if filename.suffix == '.json' and 'index' not in filename.name]

directors = list()

for acmi_file in tqdm.tqdm(acmi_files):

    with open(acmi_file, encoding='utf-8') as acmi_data:
        acmi_data = json.load(acmi_data)

    if 'creators_primary' in acmi_data:
        for y in acmi_data['creators_primary']:
            if 'role' in y:
                if y['role'] == 'director':
                    directors.append(y['creator_id'])

print(len(pydash.uniq(directors)))

acmi_creators = pandas.read_csv(pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'tsv' / 'creators.tsv', delimiter='\t', low_memory=False)
acmi_creators = acmi_creators.loc[acmi_creators.id.isin(directors)]

print(len(acmi_creators))
acmi_creators.sample(10)

100%|██████████| 42920/42920 [01:00<00:00, 709.14it/s]


7548
3484


,id,name,also_known_as,date_of_birth,date_of_death,places_of_operation,biography,biography_author,date_of_biography,external_links,uuid,source,source_identifier,external_references,date_modified
7130,82175,Rolf Lyssy,NaN,NaN,NaN,Switzerland,NaN,NaN,2023-05-09T15:08:40.825119+10:00,https://en.wikipedia.org/wiki/Rolf_Lyssy,debeaa26-523c-40bc-a6d3-b050ff4a76f3,Vernon,30904.0,"(Wikidata,Q117087),(VIAF,66734466),(LOC Auth,n...",2023-06-14T16:33:30.576250+10:00
12733,31135,Gary Halvorson,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T19:19:37.802580+10:00,NaN,a12591b9-01f3-4cd9-8853-6c9f9265e2ed,Vernon,19406.0,NaN,2023-06-14T16:33:30.576250+10:00
1283,19433,Mike Newell,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T16:12:41.996503+10:00,NaN,c480adb8-0ff7-4ea1-b109-165582e3afe0,Vernon,25384.0,NaN,2023-06-14T16:33:30.576250+10:00
525,24539,Paul Czinner,NaN,1890,1972,NaN,NaN,NaN,2023-05-09T19:52:14.835838+10:00,NaN,f6b7c755-4476-4795-bf52-07ca0ec70233,Vernon,26200.0,NaN,2023-06-14T16:33:30.576250+10:00
13336,30136,Paul McGuiaan,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T19:32:47.385237+10:00,NaN,86839afb-6aa8-481e-a091-d3b1f69012c7,Vernon,26289.0,NaN,2023-06-14T16:33:30.576250+10:00
9253,27232,Bo-Erik Gyberg,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T18:20:50.280739+10:00,NaN,76a47eee-52ca-4d92-96c0-c1bd4ae352a4,Vernon,16125.0,NaN,2023-06-14T16:33:30.576250+10:00
18986,79379,Shane Luther,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T16:31:09.196268+10:00,NaN,9bae80ec-e784-4fd2-b062-695f1b3bf148,Vernon,28709.0,NaN,2023-06-14T16:33:30.576250+10:00
8449,77239,Daniel Petrie,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T13:36:45.441419+10:00,NaN,8b33b133-c680-4bd0-9236-f04493d9ee81,Vernon,17404.0,NaN,2023-06-14T16:33:30.576250+10:00
7805,82746,Zhai Junjie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f4e0da64-7ca4-4387-8fe9-3cbd3d673419,Vernon,NaN,NaN,2023-06-14T16:33:30.576250+10:00
9283,21577,Stewart Main,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T19:24:45.628559+10:00,NaN,ea349735-032a-4cac-90aa-498c2347c837,Vernon,29226.0,NaN,2023-06-14T16:33:30.576250+10:00
